# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [35]:
# import libraries
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])

import re
import numpy as np
import pandas as pd
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alien\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alien\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from sqlalchemy   import create_engine
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponse",con=engine)

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df[df.related==2]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
117,146,Dans la zone de Saint Etienne la route de Jacm...,Nan zon st. etine rout jakmel la bloke se mize...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
219,263,. .. i with limited means. Certain patients co...,t avec des moyens limites. Certains patients v...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305,373,The internet caf Net@le that's by the Dal road...,Cyber cafe net@le ki chita rout de dal tou pr ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
460,565,"Bonsoir, on est a bon repos aprs la compagnie ...",Bonswa nou nan bon repo apri teleko nan wout t...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
576,700,URGENT CRECHE ORPHANAGE KAY TOUT TIMOUN CROIX ...,r et Salon Furterer. mwen se yon Cosmtologue. ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
655,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
882,1063,no authority has passed by to see us. We don't...,EN OKENN OTORITE POKO PASE WE NOU NOU PAGEN KO...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
896,1080,It's Over in Gressier. The population in the a...,Se over toujou nan gresye 24e seksyon komin. P...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
924,1113,we sleep with the baby. Thanks in advance for ...,sa nou demi avek ti bebe. Mesi davans pou d no...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
930,1121,I need help in Jrmie because I was in Port-au-...,MWEN bezwen ed lan jeremi paske mwen te portop...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df.message
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [6]:
Y = df.iloc[:,4:]
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
Y.iloc[3]

related                   1
request                   1
offer                     0
aid_related               1
medical_help              0
medical_products          1
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 1
infrastructure_related    1
transport                 0
buildings                 1
electricity               0
tools                     0
hospitals                 1
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

In [8]:
# load data from database
from sqlalchemy   import create_engine

engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponse",con=engine)
X = df.message
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

may helpful! https://stackoverflow.com/questions/46770088/multioutput-classifier-learning-5-target-variables


In [10]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [11]:
pipeline.get_params()


{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x0000023247DBB730>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
               oob_score=Fal

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
 X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(19662,)
(6554,)
(19662, 36)
(6554, 36)


In [14]:
pipeline.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
prediction = pipeline.predict(X_test)

In [16]:
category_predict = prediction
y_test_cate = y_test

In [17]:
y_test_cate[y_test_cate.related==2]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
924,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12057,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12128,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12307,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12152,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
460,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7549,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11433,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
882,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5550,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
category_predict_df = pd.DataFrame(data =category_predict,columns = y_test_cate.columns,index =y_test_cate.index  )

In [19]:
category_predict_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
4389,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2464,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
13343,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
17880,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
555,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
category_predict_df[category_predict_df.related==2]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
12057,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12307,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8234,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12254,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12278,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8376,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3106,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10383,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
12194,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12275,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
from sklearn.metrics import classification_report
column_name = list(category_predict_df)

for column in column_name:
    print("_{}_ category socre:\n".format(column))
    print(classification_report(category_predict_df[column], y_test_cate[column]))

_related_ category socre:

              precision    recall  f1-score   support

           0       0.36      0.61      0.46       891
           1       0.93      0.82      0.87      5648
           2       0.22      0.60      0.32        15

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.50      0.68      0.55      6554
weighted avg       0.85      0.79      0.82      6554

_request_ category socre:

              precision    recall  f1-score   support

           0       0.98      0.88      0.93      6049
           1       0.36      0.82      0.50       505

   micro avg       0.87      0.87      0.87      6554
   macro avg       0.67      0.85      0.71      6554
weighted avg       0.94      0.87      0.89      6554

_offer_ category socre:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554
           1       0.00      0.00      0.00         0

   micro avg       1.00      1.00      1.00      

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [23]:
from sklearn.model_selection import GridSearchCV
parameters =  {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.75, 1.0),
#       'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
#        'clf__estimator__n_estimators': [50, 100, 200],
        
        
        
    }

cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs = -1)
cv.fit(X_train,y_train)
predict_cv = cv.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\s

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
Y_pred_cv = pd.DataFrame(predict_cv, columns = column_name)

In [25]:
for i in range(36):
    print(column_name[i],\
          '\n',\
          classification_report(y_test_cate.iloc[:,i], Y_pred_cv.iloc[:,i]))

related 
               precision    recall  f1-score   support

           0       0.60      0.38      0.46      1511
           1       0.83      0.92      0.87      5002
           2       0.62      0.24      0.35        41

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.68      0.51      0.56      6554
weighted avg       0.77      0.79      0.77      6554

request 
               precision    recall  f1-score   support

           0       0.88      0.98      0.93      5393
           1       0.80      0.39      0.52      1161

   micro avg       0.87      0.87      0.87      6554
   macro avg       0.84      0.68      0.73      6554
weighted avg       0.87      0.87      0.86      6554

offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [26]:
for i in range(36):
    category = column_name[i]
    accuracy = accuracy_score(y_test_cate.iloc[:,i], Y_pred_cv.iloc[:,i])
    precision = precision_score(y_test_cate.iloc[:,i], Y_pred_cv.iloc[:,i], average='micro')
    recall = recall_score(y_test_cate.iloc[:,i], Y_pred_cv.iloc[:,i], average='micro')
    f1 = f1_score(y_test_cate.iloc[:,i], Y_pred_cv.iloc[:,i], average='micro')
    print(category)
    print("\t Accuracy: %.4f \t Precision: %.4f \t Recall: %.4f \t F1-Score: %.4f \n" %\
              (accuracy, precision, recall, f1))

related
	 Accuracy: 0.7919 	 Precision: 0.7919 	 Recall: 0.7919 	 F1-Score: 0.7919 

request
	 Accuracy: 0.8744 	 Precision: 0.8744 	 Recall: 0.8744 	 F1-Score: 0.8744 

offer
	 Accuracy: 0.9960 	 Precision: 0.9960 	 Recall: 0.9960 	 F1-Score: 0.9960 

aid_related
	 Accuracy: 0.7228 	 Precision: 0.7228 	 Recall: 0.7228 	 F1-Score: 0.7228 

medical_help
	 Accuracy: 0.9211 	 Precision: 0.9211 	 Recall: 0.9211 	 F1-Score: 0.9211 

medical_products
	 Accuracy: 0.9519 	 Precision: 0.9519 	 Recall: 0.9519 	 F1-Score: 0.9519 

search_and_rescue
	 Accuracy: 0.9738 	 Precision: 0.9738 	 Recall: 0.9738 	 F1-Score: 0.9738 

security
	 Accuracy: 0.9815 	 Precision: 0.9815 	 Recall: 0.9815 	 F1-Score: 0.9815 

military
	 Accuracy: 0.9677 	 Precision: 0.9677 	 Recall: 0.9677 	 F1-Score: 0.9677 

child_alone
	 Accuracy: 1.0000 	 Precision: 1.0000 	 Recall: 1.0000 	 F1-Score: 1.0000 

water
	 Accuracy: 0.9510 	 Precision: 0.9510 	 Recall: 0.9510 	 F1-Score: 0.9510 

food
	 Accuracy: 0.9310 	 Precision

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
# use xgboost as the classifier
xgb_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('xgb_clf', MultiOutputClassifier(XGBClassifier()))
])

In [29]:
xgb_pipeline.fit(X_train,y_train)
xgb_prediction = xgb_pipeline.predict(X_test)

In [32]:
Y_pred_xgb = pd.DataFrame(xgb_prediction, columns = column_name)

In [33]:
for i in range(36):
    category = column_name[i]
    accuracy = accuracy_score(y_test_cate.iloc[:,i], Y_pred_xgb.iloc[:,i])
    precision = precision_score(y_test_cate.iloc[:,i], Y_pred_xgb.iloc[:,i], average='micro')
    recall = recall_score(y_test_cate.iloc[:,i], Y_pred_xgb.iloc[:,i], average='micro')
    f1 = f1_score(y_test_cate.iloc[:,i], Y_pred_xgb.iloc[:,i], average='micro')
    print(category)
    print("\t Accuracy: %.4f \t Precision: %.4f \t Recall: %.4f \t F1-Score: %.4f \n" %\
              (accuracy, precision, recall, f1))

related
	 Accuracy: 0.7992 	 Precision: 0.7992 	 Recall: 0.7992 	 F1-Score: 0.7992 

request
	 Accuracy: 0.8944 	 Precision: 0.8944 	 Recall: 0.8944 	 F1-Score: 0.8944 

offer
	 Accuracy: 0.9960 	 Precision: 0.9960 	 Recall: 0.9960 	 F1-Score: 0.9960 

aid_related
	 Accuracy: 0.7473 	 Precision: 0.7473 	 Recall: 0.7473 	 F1-Score: 0.7473 

medical_help
	 Accuracy: 0.9280 	 Precision: 0.9280 	 Recall: 0.9280 	 F1-Score: 0.9280 

medical_products
	 Accuracy: 0.9587 	 Precision: 0.9587 	 Recall: 0.9587 	 F1-Score: 0.9587 

search_and_rescue
	 Accuracy: 0.9756 	 Precision: 0.9756 	 Recall: 0.9756 	 F1-Score: 0.9756 

security
	 Accuracy: 0.9821 	 Precision: 0.9821 	 Recall: 0.9821 	 F1-Score: 0.9821 

military
	 Accuracy: 0.9704 	 Precision: 0.9704 	 Recall: 0.9704 	 F1-Score: 0.9704 

child_alone
	 Accuracy: 1.0000 	 Precision: 1.0000 	 Recall: 1.0000 	 F1-Score: 1.0000 

water
	 Accuracy: 0.9667 	 Precision: 0.9667 	 Recall: 0.9667 	 F1-Score: 0.9667 

food
	 Accuracy: 0.9582 	 Precision

### 9. Export your model as a pickle file

In [36]:
filename = 'xgb_classifier.pkl'
pickle.dump(xgb_pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.